# Collecte des données footballistiques

### Configuration de Selenium et Lancement du Site Web

In [51]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

website = "https://fbref.com/en/comps/9/2024-2025/2024-2025-Premier-League-Stats"
path = "../chromedriver-win32/chromedriver.exe"
service = Service(path)
driver = webdriver.Chrome(service=service)

driver.get(website) 



### Extraction des Équipes et de leurs Liens depuis la Page Web

In [52]:
team_data = []
team_elements = driver.find_elements(By.CSS_SELECTOR,'td[data-stat="team"] > a')
seen = set()

for t in team_elements:
    name = t.text.strip()
    href = t.get_attribute('href')
    
    if name and name not in seen:
        team_data.append((name, href))
        seen.add(name)

print(team_data)

[('Liverpool', 'https://fbref.com/en/squads/822bd0ba/2024-2025/Liverpool-Stats'), ('Arsenal', 'https://fbref.com/en/squads/18bb7c10/2024-2025/Arsenal-Stats'), ('Manchester City', 'https://fbref.com/en/squads/b8fd03ef/2024-2025/Manchester-City-Stats'), ('Chelsea', 'https://fbref.com/en/squads/cff3d9bb/2024-2025/Chelsea-Stats'), ('Newcastle Utd', 'https://fbref.com/en/squads/b2b47a98/2024-2025/Newcastle-United-Stats'), ('Aston Villa', 'https://fbref.com/en/squads/8602292d/2024-2025/Aston-Villa-Stats'), ("Nott'ham Forest", 'https://fbref.com/en/squads/e4a775cb/2024-2025/Nottingham-Forest-Stats'), ('Brighton', 'https://fbref.com/en/squads/d07537b9/2024-2025/Brighton-and-Hove-Albion-Stats'), ('Bournemouth', 'https://fbref.com/en/squads/4ba7cbea/2024-2025/Bournemouth-Stats'), ('Brentford', 'https://fbref.com/en/squads/cd051869/2024-2025/Brentford-Stats'), ('Fulham', 'https://fbref.com/en/squads/fd962109/2024-2025/Fulham-Stats'), ('Crystal Palace', 'https://fbref.com/en/squads/47c64c55/2024-2